In [ ]:
import panel as pn
pn.extension('vtk')

The ``VTKSynchronized`` pane renders vtk scene inside a panel, making it possible to interact with complex geometries in 3D.
It allows to keep a synchronize state between the ``vtkRenderWindow`` defines on python side and the one diplayed in the pane through vtk-js.
Python is playing the role of a server and send informations about the scene to the client.
The synchronization is done only on one side: Python => javascript. Modifications done on javascript side are not reflected to the Python ``vtkRenderWindow``

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

* **``axes``** (dict): A dictionary with the parameters of the axes to construct in the 3d view.
    Must contain at least ``xticker``, ``yticker`` and ``zticker``.
    - ``[x|y|z]ticker`` is a dictionary which contains:
        - ``ticks`` (array of numbers) - required. Positions in the scene coordinates
        of the coresponding axe ticks
        - ``labels`` (array of strings) - optional. Label displayed respectively to
        the `ticks` positions.

        If `labels` are not defined they are infered from the `ticks` array.
    - ``digits``: number of decimal digits when `ticks` are converted to `labels`.
    - ``fontsize``: size in pts of the ticks labels.
    - ``show_grid``: boolean. If true (default) the axes grid is visible.
    - ``grid_opacity``: float between 0-1. Defines the grid opacity.
    - ``axes_opacity``: float between 0-1. Defines the axes lines opacity.
* **``camera``** (dict): A dictionary reflecting the current state of the VTK camera
* **``enable_keybindings``** (bool): A boolean to activate/deactivate keybindings. Bound keys are:
  - s: set representation of all actors to *surface*
  - w: set representation of all actors to *wireframe*
  - v: set representation of all actors to *vertex*
  - r: center the actors and move the camera so that all actors are visible
  
  The mouse must be over the pane to work
  <br>**Warning**: These keybindings may not work as expected in a notebook context, if they interact with already bound keys
* **``orientation_widget``** (bool): A boolean to activate/deactivate the orientation widget in the 3D pane. This widget is clickable and allows to rotate the scene in one of the orthographic projections.
* **``object``** (object): Must be a ``vtkRenderWindow`` instance.

#### Properties:

* **``actors``** : return the list of the vtkActors in the scene

* **``vtk_camera``** :  return the vtkCamera of the renderer hold by the panel

#### Methods:

Several helpers to work with the vtk scene are exposed.

* **``set_background(r: float, g: float, b: float)``** : Set the color of the scene background as rgb color where `r`, `g`, and `b` are float in (0, 1)

* **``synchronize()``** : Sychronize the modifications of the ``vtkRenderWindow``

* **``unlink_camera()``** : Create a new vtkCamera object, allowing to the panel to have it's own camera.

* **``link_camera(other: VTKSynchronized)``** : Both panel will share the same camera, any change in one panel camera will be applied in the link panel

**Warning** for applying any modification generated by these methods, the `sychronize()` method must be called
___

## Rendering VTK objects

There are slight differences with classical code generally used in ``VTK``. As rendering of the objects and interactions with the view are handle by the VTK panel, we don't need to make a call to the `Render` method of the `vtkRenderWindow` (which would pop up the classical VTK window) or to specify a `vtkRenderWindowInteractor`. 

In [ ]:
import vtk
from vtk.util.colors import tomato

# This creates a polygonal cylinder model with eight circumferential
# facets.
cylinder = vtk.vtkCylinderSource()
cylinder.SetResolution(8)

# The mapper is responsible for pushing the geometry into the graphics
# library. It may also do color mapping, if scalars or other
# attributes are defined.
cylinderMapper = vtk.vtkPolyDataMapper()
cylinderMapper.SetInputConnection(cylinder.GetOutputPort())

# The actor is a grouping mechanism: besides the geometry (mapper), it
# also has a property, transformation matrix, and/or texture map.
# Here we set its color and rotate it -22.5 degrees.
cylinderActor = vtk.vtkActor()
cylinderActor.SetMapper(cylinderMapper)
cylinderActor.GetProperty().SetColor(tomato)
# We must set ScalarVisibilty to 0 to use tomato Color
cylinderMapper.SetScalarVisibility(0)
cylinderActor.RotateX(30.0)
cylinderActor.RotateY(-45.0)

# Create the graphics structure. The renderer renders into the render
# window.
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)

# Add the actors to the renderer, set the background and size
ren.AddActor(cylinderActor)
ren.SetBackground(0.1, 0.2, 0.4)

geom_pane = pn.pane.VTKSynchronized(renWin, width=500, height=500)

geom_pane

We can also add additional actors to the plot call `synchronize` method to update the panel:

In [ ]:
sphere = vtk.vtkSphereSource()

sphereMapper = vtk.vtkPolyDataMapper()
sphereMapper.SetInputConnection(sphere.GetOutputPort())

sphereActor = vtk.vtkActor()
sphereActor.SetMapper(sphereMapper)
sphereActor.GetProperty().SetColor(tomato)
sphereMapper.SetScalarVisibility(0)
sphereActor.SetPosition(0.5, 0.5, 0.5)

ren.AddActor(sphereActor)

geom_pane.synchronize()

### Working with [`Pyvista`](https://docs.pyvista.org/)


In most examples of the documentation, [Pyvista](https://docs.pyvista.org/) library will be used.

Even if most of examples can be rewrite with only vtk dependency, `pyvista` allows pythonic and concise syntax and expose main features needed to work with VTK objects

The above example can be rewrite with the following few steps:

 - The link between pyvista and panel is done by the following code.

In [ ]:
import pyvista as pv

plotter = pv.Plotter() # we define a pyvista plotter
plotter.background_color = (0.1, 0.2, 0.4)
# we create a `VTKSynchronized` panel around the render window
vtkpan = pn.panel(plotter.ren_win, width=500, height=500) 

 - We create and add some objects to the pysvita plotter and we displayed them using the vtk panel

In [ ]:
pvcylinder = pv.Cylinder(resolution=8)
cylinder_actor = plotter.add_mesh(pvcylinder, color=tomato, smooth_shading=True)
cylinder_actor.RotateX(30.0)
cylinder_actor.RotateY(-45.0)
vtkpan

 - Now we add the sphere to the scene using pyvista plotter and synchronize the vtkpan

In [ ]:
sphere_actor = plotter.add_mesh(pv.Sphere(
    theta_resolution=8, phi_resolution=8,
    center=(0.5, 0.5, 0.5)),color=tomato, smooth_shading=True
)
sphere_actor.RotateX(30.0)
sphere_actor.RotateY(-45.0)
vtkpan.synchronize()

We can reset the camera to see all actors in the scene

In [ ]:
vtkpan.reset_camera()
vtkpan.synchronize()

## Advance usage and interactivity

This examples will show :
 - where information about `scalars` used to color 3d objects can be found
 - utilisation of the `orientation widget`
 - how to add `axes` in the panel
 - collaboration with the `player` widget to create an animation

### Scene creation

First we create scene with a nice spline inspired by this examples : [Creating a Spline](https://docs.pyvista.org/examples/00-load/create-spline.html#creating-a-spline)


In [ ]:
import numpy as np

def make_points():
    """Helper to make XYZ points"""
    theta = np.linspace(-4 * np.pi, 4 * np.pi, 100)
    z = np.linspace(-2, 2, 100)
    r = z**2 + 1
    x = r * np.sin(theta)
    y = r * np.cos(theta)
    return np.column_stack((x, y, z))

def lines_from_points(points):
    """Given an array of points, make a line set"""
    poly = pv.PolyData()
    poly.points = points
    cells = np.full((len(points)-1, 3), 2, dtype=np.int)
    cells[:, 1] = np.arange(0, len(points)-1, dtype=np.int)
    cells[:, 2] = np.arange(1, len(points), dtype=np.int)
    poly.lines = cells
    return poly

points = make_points()
line = lines_from_points(points)
line["scalars"] = np.arange(line.n_points) # By default pyvista use viridis colormap
tube = line.tube(radius=0.1) #=> the object we will represent in the scene


pl = pv.Plotter()
pl.camera_position =  [(4.440892098500626e-16, -21.75168228149414, 4.258553981781006),
                       (4.440892098500626e-16, 0.8581731039809382, 0),
                       (0, 0.1850949078798294, 0.982720673084259)]

pl.add_mesh(tube, smooth_shading=True)
pan = pn.panel(pl.ren_win, width=500, orientation_widget=True)
pan

By clicking on the three dots colobar can displayed

The orientation widget on the bottom right of the panel is clickable too and allow to orient the camera in a specific plan

### Adding axes to the VTK scene

Using the `axes` parameter, it is possible to add an axes into the 3d view.

`Axes` are a like `orientation widget` and `colorbar`, a specific object built on javascript side.

So they will not be synchronized with the python `vtkRenderWindow`

In [ ]:
axes = dict(
    origin = [-5, 5, -2],
    xticker = {'ticks': np.linspace(-5,5,5)},
    yticker = {'ticks': np.linspace(-5,5,5)},
    zticker = {'ticks': np.linspace(-2,2,5), 'labels': [''] + [str(int(item)) for item in np.linspace(-2,2,5)[1:]]},
    fontsize = 12,
    digits = 1,
    grid_opacity = 0.5,
    show_grid=True
)
pan.axes = axes

### Create an animation

For this animation we will rotate the actor and roll the scalars on the spline tube

In [ ]:
pan_clone = pan.clone() # we clone the panel to animate only this panel
pan_clone.unlink_camera() # we don't want to share the camera with the previous panel
player = pn.widgets.Player(name='Player', start=0, end=100, value=0, loop_policy='loop', interval=100)
scalars = tube["scalars"]
@pn.depends(value=player.param.value)
def animate(value):
    tube["scalars"] = np.roll(scalars, 200*value)
    pan_clone.actors[0].SetOrientation(0, 0, -20*value)
    pan_clone.synchronize()
pn.Column(pan_clone, player, animate).servable()